## SARIMAX Modelling with Exogenous variable + log transformation + Catboost Regressor - 11th July 2019 to 15th Dec 2019--Version 3

In [118]:
# Importing necessary packages
import numpy as np
import pandas as pd
import math as m
import time
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sb
import datetime as dt
from itertools import product
from collections import Counter
from matplotlib.pylab import rcParams
from catboost import CatBoostRegressor,Pool
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from statistics import stdev
import re
from category_encoders import *
from datetime import timedelta
from sklearn.metrics import mean_squared_error,mean_absolute_error

# Importing ARIMA packages

from statsmodels.tsa.arima_model import ARIMA,ARMA,ARIMAResults,ARMAResults
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from pmdarima import auto_arima

# Importing SARIMA packages

from statsmodels.tsa.statespace.sarimax import SARIMAX

In [6]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to C:\Users\Anindam
[nltk_data]     Som\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [120]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

# To visualise all the columns in a dataframe
pd.pandas.set_option('display.max_columns', None)

# Setting maximum row numbers
pd.set_option('display.max_rows', 1000)

In [264]:
# Importing the data

data = pd.read_csv('/jup/AS/Dispatch_Forecasting/SARIMAX_Original.csv')

# Dropping unnecessary features
data.drop('Unnamed: 0',axis = 1, inplace = True)

data['DATE_TZ'] = pd.to_datetime(data['DATE_TZ'])

#data = data.loc[data['DATE_TZ']> '2019-07-10']
data.head()

,DATE_TZ,CAPACITY_BUCKET_ID,CATEGORY_ID,VOLUME
0,2019-07-10,300851.0,5.0,0.0
1,2019-07-10,300851.0,6.0,0.0
2,2019-07-10,300851.0,7.0,0.0
3,2019-07-10,300852.0,5.0,0.0
4,2019-07-10,300852.0,6.0,0.0


In [278]:
data.shape

(280296, 7)

In [241]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280296 entries, 0 to 280295
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   DATE_TZ             280296 non-null  datetime64[ns]
 1   CAPACITY_BUCKET_ID  280296 non-null  float64       
 2   CATEGORY_ID         280296 non-null  float64       
 3   VOLUME              280296 non-null  float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 8.6 MB


In [148]:
"""
# Adding new row
start_lower = 0
end_lower = data.shape[0]
lower_half = [*range(start_lower, end_lower, 1)]
lower_half = [x.__add__(1) for x in lower_half]
index_ = lower_half
data.index = index_ 
data.loc[0] = ['2019-07-10',300851.0,6.0,0.0]
data = data.sort_index() 
"""

"\n# Adding new row\nstart_lower = 0\nend_lower = data.shape[0]\nlower_half = [*range(start_lower, end_lower, 1)]\nlower_half = [x.__add__(1) for x in lower_half]\nindex_ = lower_half\ndata.index = index_ \ndata.loc[0] = ['2019-07-10',300851.0,6.0,0.0]\ndata = data.sort_index() \n"

In [13]:
# Number of unique bucket ids
print(len(data['CAPACITY_BUCKET_ID'].unique()))

458


In [235]:
min_date = data['DATE_TZ'].min()
min_date

Timestamp('2019-07-10 00:00:00')

In [236]:
max_date = data['DATE_TZ'].max()
max_date

Timestamp('2020-01-29 00:00:00')

In [265]:
train_start_date = data['DATE_TZ'].min()
train_end_date = data['DATE_TZ'].max()
#test_start_date = data_test['DATE_TZ'].min()
#test_end_date = data_test['DATE_TZ'].max() 

In [266]:
print(train_start_date)
print(train_end_date)
#print(test_start_date)
#print(test_end_date)

2019-07-10 00:00:00
2020-01-29 00:00:00


In [135]:
# Introducing Exogenous variables - 'Isholiday' to train and test set

# Creating a feature 'Isholiday'
def holiday(date):
    day=date.day
    month=date.month
    if (day,month) in [(1,1),(25,5),(4,7),(1,5),(26,11),(25,12)]:
        return 1.0
    else:
        return 0.0

In [267]:
# Setting up holiday feature
data['IsHoliday'] = 0

In [268]:
data['IsHoliday'] = data["DATE_TZ"].apply(holiday)

In [269]:
data.loc[:,'dow'] = data['DATE_TZ'].dt.weekday

In [270]:
data.loc[:,'Is_weekend'] = np.where(data['dow'].isin([5,6]),1,0)

In [271]:
data['VOLUME_AVG'] = 0
data['VOLUME_AVG'] = data.groupby(['CAPACITY_BUCKET_ID','CATEGORY_ID','dow'])['VOLUME'].transform('mean')
data.head()

,DATE_TZ,CAPACITY_BUCKET_ID,CATEGORY_ID,VOLUME,IsHoliday,dow,Is_weekend,VOLUME_AVG
0,2019-07-10,300851.0,5.0,0.0,0.0,2,0,15.900000
1,2019-07-10,300851.0,6.0,0.0,0.0,2,0,1.600000
2,2019-07-10,300851.0,7.0,0.0,0.0,2,0,12.100000
3,2019-07-10,300852.0,5.0,0.0,0.0,2,0,7.200000
4,2019-07-10,300852.0,6.0,0.0,0.0,2,0,0.533333


In [250]:
# Checking for outliers
data.describe(percentiles = [.1,.2,.3,.4,.5,.6,.7,.8,.9])

,CAPACITY_BUCKET_ID,CATEGORY_ID,VOLUME,IsHoliday,dow,Is_weekend,VOLUME_AVG
count,280296.000000,280296.000000,280296.000000,280296.000000,280296.000000,280296.000000,280296.000000
mean,304272.879913,6.000000,8.268719,0.014706,2.995098,0.284314,8.268719
std,3682.619184,0.816498,12.228596,0.120373,1.996318,0.451088,11.051922
min,300851.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10%,300896.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.241379
20%,301602.000000,5.000000,0.000000,0.000000,1.000000,0.000000,0.620690
30%,302149.000000,5.000000,1.000000,0.000000,2.000000,0.000000,1.200000
40%,302195.000000,6.000000,2.000000,0.000000,2.000000,0.000000,2.034483
50%,302906.500000,6.000000,3.000000,0.000000,3.000000,0.000000,3.413793
60%,303650.000000,6.000000,5.000000,0.000000,4.000000,0.000000,5.482759


In [272]:
# Setting the index
data.index = data['DATE_TZ']

In [273]:
# Dropping unnecessary features
data.drop('DATE_TZ',axis = 1,inplace = True)

In [274]:
# Applying log transformation

data['VOLUME'] = np.log(data['VOLUME']+1)

In [102]:
"""
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

cols = ['VOLUME','dow']

features = data_train[cols]

ct = ColumnTransformer([('scaler', StandardScaler(), ['VOLUME','dow'])], remainder='passthrough')
        
data_train = ct.fit_transform(features)

"""

In [275]:
data.shape

(280296, 7)

In [276]:
buck_id = data['CAPACITY_BUCKET_ID'].unique()
cat_id = data['CATEGORY_ID'].unique()
print(len(buck_id),len(cat_id))

458 3


## Training

In [277]:
#Training for all Bucket Ids
buck_id = data['CAPACITY_BUCKET_ID'].unique()
cat_id = data['CATEGORY_ID'].unique()
column_names = ['CAPACITY_BUCKET_ID','CATEGORY_ID','y_hat']
data_predict = pd.DataFrame(columns = column_names)
data_result = pd.DataFrame()

# Training started
for b_id in buck_id:
    for c_id in cat_id:
        
        data_sarima = data[(data['CAPACITY_BUCKET_ID'] == b_id) & (data['CATEGORY_ID'] == c_id)]

        sarima = SARIMAX(data_sarima['VOLUME'],exog = data_sarima[['IsHoliday','dow','Is_weekend','VOLUME_AVG']],order=(4,1,4),freq = 'D',seasonal_order=(2, 0, [1, 2], 5),enforce_stationarity=False, enforce_invertibility=False).fit()
                                 
        pred = sarima.predict(train_start_date,train_end_date,exog = data[['IsHoliday','dow','Is_weekend','VOLUME_AVG']])[1:]
        
        inversed = np.exp(pred) #Inverse Logarithmic transformation

        data_predict['y_hat'] = inversed
        
        data_predict['CAPACITY_BUCKET_ID'] = b_id

        data_predict['CATEGORY_ID'] = c_id 
        
        data_result = data_result.append(data_predict,sort = True)
        
        data_predict = data_predict[0:0]  # Resetting the dataframe

data_result1 = data_result.loc[data_result.index > '2019-07-10']   # Filtering the dataframe
print(data_result1.shape)

(278922, 3)


In [279]:
# Sorting the dataset
data_result1 = data_result1.sort_index(axis = 0,ascending=True)
data_result1['DATE_TZ'] = 0
data_result1['DATE_TZ'] = data_result1.index
data_result1.head()

,CAPACITY_BUCKET_ID,CATEGORY_ID,y_hat,DATE_TZ
2019-07-11,300851.0,5.0,1.414030,2019-07-11
2019-07-11,302160.0,6.0,1.032582,2019-07-11
2019-07-11,302138.0,7.0,1.116294,2019-07-11
2019-07-11,302915.0,5.0,1.404260,2019-07-11
2019-07-11,301587.0,6.0,0.999142,2019-07-11


In [280]:
#Checking for Null values
data_result1.isnull().sum()

CAPACITY_BUCKET_ID    0
CATEGORY_ID           0
y_hat                 0
DATE_TZ               0
dtype: int64

In [281]:
# Altering the original dataframe
data['VOLUME'] = np.exp(data['VOLUME']) 
data['VOLUME'] = (data['VOLUME'] - 1)
data.head()

,CAPACITY_BUCKET_ID,CATEGORY_ID,VOLUME,IsHoliday,dow,Is_weekend,VOLUME_AVG
DATE_TZ,,,,,,,
2019-07-10,300851.0,5.0,0.0,0.0,2,0,15.900000
2019-07-10,300851.0,6.0,0.0,0.0,2,0,1.600000
2019-07-10,300851.0,7.0,0.0,0.0,2,0,12.100000
2019-07-10,300852.0,5.0,0.0,0.0,2,0,7.200000
2019-07-10,300852.0,6.0,0.0,0.0,2,0,0.533333


In [282]:
#Introducing new column
data['DATE_TZ'] = 0
data['DATE_TZ'] = data.index

In [283]:
data = data.loc[data['DATE_TZ'] > '2019-07-10']
print(data.shape)
data.head()

(278922, 8)


,CAPACITY_BUCKET_ID,CATEGORY_ID,VOLUME,IsHoliday,dow,Is_weekend,VOLUME_AVG,DATE_TZ
DATE_TZ,,,,,,,,
2019-07-11,300851.0,5.0,23.0,0.0,3,0,17.172414,2019-07-11
2019-07-11,300851.0,6.0,1.0,0.0,3,0,1.689655,2019-07-11
2019-07-11,300851.0,7.0,19.0,0.0,3,0,14.586207,2019-07-11
2019-07-11,300852.0,5.0,9.0,0.0,3,0,7.068966,2019-07-11
2019-07-11,300852.0,6.0,1.0,0.0,3,0,0.793103,2019-07-11


In [284]:
# Merging the dataframes
merge = data_result1.merge(data,how="left",on=['CAPACITY_BUCKET_ID','CATEGORY_ID'])

In [285]:
print(merge.shape)
merge.head()

(56621166, 10)


,CAPACITY_BUCKET_ID,CATEGORY_ID,y_hat,DATE_TZ_x,VOLUME,IsHoliday,dow,Is_weekend,VOLUME_AVG,DATE_TZ_y
0,300851.0,5.0,1.41403,2019-07-11,23.0,0.0,3,0,17.172414,2019-07-11
1,300851.0,5.0,1.41403,2019-07-11,24.0,0.0,4,0,17.172414,2019-07-12
2,300851.0,5.0,1.41403,2019-07-11,11.0,0.0,5,1,11.068966,2019-07-13
3,300851.0,5.0,1.41403,2019-07-11,7.0,0.0,6,1,5.965517,2019-07-14
4,300851.0,5.0,1.41403,2019-07-11,29.0,0.0,0,0,20.103448,2019-07-15


In [286]:
# Removing Duplicate records
merge1 = merge.drop_duplicates(['CAPACITY_BUCKET_ID','CATEGORY_ID','y_hat','DATE_TZ_x'])
print(merge1.shape)
merge1.head()

(278922, 10)


,CAPACITY_BUCKET_ID,CATEGORY_ID,y_hat,DATE_TZ_x,VOLUME,IsHoliday,dow,Is_weekend,VOLUME_AVG,DATE_TZ_y
0,300851.0,5.0,1.414030,2019-07-11,23.0,0.0,3,0,17.172414,2019-07-11
203,302160.0,6.0,1.032582,2019-07-11,1.0,0.0,3,0,1.137931,2019-07-11
406,302138.0,7.0,1.116294,2019-07-11,4.0,0.0,3,0,1.172414,2019-07-11
609,302915.0,5.0,1.404260,2019-07-11,12.0,0.0,3,0,15.068966,2019-07-11
812,301587.0,6.0,0.999142,2019-07-11,0.0,0.0,3,0,0.448276,2019-07-11


In [288]:
# Resetting the index
merge1 = merge1.reset_index()
print(merge1.shape)
merge1.head()

(278922, 11)


,index,CAPACITY_BUCKET_ID,CATEGORY_ID,y_hat,DATE_TZ_x,VOLUME,IsHoliday,dow,Is_weekend,VOLUME_AVG,DATE_TZ_y
0,0,300851.0,5.0,1.414030,2019-07-11,23.0,0.0,3,0,17.172414,2019-07-11
1,203,302160.0,6.0,1.032582,2019-07-11,1.0,0.0,3,0,1.137931,2019-07-11
2,406,302138.0,7.0,1.116294,2019-07-11,4.0,0.0,3,0,1.172414,2019-07-11
3,609,302915.0,5.0,1.404260,2019-07-11,12.0,0.0,3,0,15.068966,2019-07-11
4,812,301587.0,6.0,0.999142,2019-07-11,0.0,0.0,3,0,0.448276,2019-07-11


In [289]:
# Dropping unnecessary features
merge1.drop(['index','DATE_TZ_y'],axis = 1,inplace = True)

# Renaming the columns
merge1.loc[:,'DATE_TZ'] = merge1['DATE_TZ_x']

In [291]:
# Dropping unnecessary features
merge1.drop(['DATE_TZ_x'],axis = 1,inplace = True)
merge1.head()

,CAPACITY_BUCKET_ID,CATEGORY_ID,y_hat,VOLUME,IsHoliday,dow,Is_weekend,VOLUME_AVG,DATE_TZ
0,300851.0,5.0,1.414030,23.0,0.0,3,0,17.172414,2019-07-11
1,302160.0,6.0,1.032582,1.0,0.0,3,0,1.137931,2019-07-11
2,302138.0,7.0,1.116294,4.0,0.0,3,0,1.172414,2019-07-11
3,302915.0,5.0,1.404260,12.0,0.0,3,0,15.068966,2019-07-11
4,301587.0,6.0,0.999142,0.0,0.0,3,0,0.448276,2019-07-11


In [193]:
# Dropping unnecessary features
#merge1.drop(['level_0'],axis = 1,inplace = True)

In [292]:
merge1.index = merge1['DATE_TZ']
merge1.drop('DATE_TZ',axis = 1,inplace = True)
merge1.head()

,CAPACITY_BUCKET_ID,CATEGORY_ID,y_hat,VOLUME,IsHoliday,dow,Is_weekend,VOLUME_AVG
DATE_TZ,,,,,,,,
2019-07-11,300851.0,5.0,1.414030,23.0,0.0,3,0,17.172414
2019-07-11,302160.0,6.0,1.032582,1.0,0.0,3,0,1.137931
2019-07-11,302138.0,7.0,1.116294,4.0,0.0,3,0,1.172414
2019-07-11,302915.0,5.0,1.404260,12.0,0.0,3,0,15.068966
2019-07-11,301587.0,6.0,0.999142,0.0,0.0,3,0,0.448276


In [293]:
# Ensembling with Catboost regressor - Declaring the model
cat_model = CatBoostRegressor(iterations=5000, 
                  depth=10, learning_rate=0.005, 
                  eval_metric='MAPE',
                  border_count=254,l2_leaf_reg=1)

In [294]:
# Splitting the dataset into Training,Evaluation and Testing set

data_train = merge1.loc[merge1.index < '2019-12-16']
#data_test = merge1.loc[(merge1.index > '2019-12-15') & (merge1.index < '2020-01-16')]

# Defining ratio of Training:Evaluation set -- 80:20
eval_size = int(data_train.shape[0] * .2)
train_size = data_train.shape[0] - eval_size

cols_x = ['CAPACITY_BUCKET_ID','CATEGORY_ID','y_hat','IsHoliday','dow','Is_weekend','VOLUME_AVG']
cols_y = ['VOLUME']

# Defining X and y
X_train = data_train.iloc[:train_size,][cols_x]
X_eval = data_train.iloc[train_size:,][cols_x]
y_train = data_train.iloc[:train_size,][cols_y]
y_eval = data_train.iloc[train_size:,][cols_y]
#X_test = data_test.drop('VOLUME',axis = 1)
#y_test = data_test['VOLUME']

In [296]:
# Creating the pool
train_pool = Pool(data=X_train,label=y_train)
eval_pool = Pool(data=X_eval,label=y_eval)

# Training with Catboost
%time cat_model.fit(train_pool, plot=True, eval_set=eval_pool)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 4.1809635	test: 4.1848492	best: 4.1848492 (0)	total: 141ms	remaining: 11m 44s
1:	learn: 4.1616982	test: 4.1655639	best: 4.1655639 (1)	total: 256ms	remaining: 10m 40s
2:	learn: 4.1426027	test: 4.1464510	best: 4.1464510 (2)	total: 412ms	remaining: 11m 26s
3:	learn: 4.1238752	test: 4.1275952	best: 4.1275952 (3)	total: 511ms	remaining: 10m 38s
4:	learn: 4.1055560	test: 4.1092575	best: 4.1092575 (4)	total: 622ms	remaining: 10m 21s
5:	learn: 4.0867142	test: 4.0903975	best: 4.0903975 (5)	total: 722ms	remaining: 10m
6:	learn: 4.0679740	test: 4.0716406	best: 4.0716406 (6)	total: 839ms	remaining: 9m 58s
7:	learn: 4.0491814	test: 4.0528305	best: 4.0528305 (7)	total: 923ms	remaining: 9m 36s
8:	learn: 4.0313148	test: 4.0349462	best: 4.0349462 (8)	total: 995ms	remaining: 9m 11s
9:	learn: 4.0135840	test: 4.0171977	best: 4.0171977 (9)	total: 1.08s	remaining: 9m 1s
10:	learn: 3.9960111	test: 3.9996081	best: 3.9996081 (10)	total: 1.17s	remaining: 8m 51s
11:	learn: 3.9779348	test: 3.9815146	bes

In [311]:
# Including date column
data_test = merge1.loc[(merge1.index > '2019-12-15') & (merge1.index < '2020-01-16')]
data_test['DATE_TZ'] = 0
data_test['DATE_TZ'] = data_test.index
print(data_test.shape)
data_test.head()

(42594, 9)


,CAPACITY_BUCKET_ID,CATEGORY_ID,y_hat,VOLUME,IsHoliday,dow,Is_weekend,VOLUME_AVG,DATE_TZ
DATE_TZ,,,,,,,,,
2019-12-16,300901.0,5.0,17.400682,32.0,0.0,3,0,21.482759,2019-12-16
2019-12-16,302190.0,5.0,1.238101,0.0,0.0,3,0,0.068966,2019-12-16
2019-12-16,300871.0,6.0,1.026517,0.0,0.0,3,0,0.103448,2019-12-16
2019-12-16,300904.0,7.0,5.271912,10.0,0.0,3,0,5.103448,2019-12-16
2019-12-16,310417.0,6.0,1.205570,0.0,0.0,3,0,0.172414,2019-12-16


In [312]:
# Dropping feature 'VOLUME_AVG'
data_test.drop(['VOLUME_AVG'],axis = 1,inplace = True)
print(data_test.shape)

(42594, 8)


In [332]:
data_train['DATE_TZ'] = 0
data_train['DATE_TZ'] = data_train.index
data_train.head()

,CAPACITY_BUCKET_ID,CATEGORY_ID,y_hat,VOLUME,IsHoliday,dow,Is_weekend,VOLUME_AVG,DATE_TZ
DATE_TZ,,,,,,,,,
2019-07-11,300851.0,5.0,1.414030,23.0,0.0,3,0,17.172414,2019-07-11
2019-07-11,302160.0,6.0,1.032582,1.0,0.0,3,0,1.137931,2019-07-11
2019-07-11,302138.0,7.0,1.116294,4.0,0.0,3,0,1.172414,2019-07-11
2019-07-11,302915.0,5.0,1.404260,12.0,0.0,3,0,15.068966,2019-07-11
2019-07-11,301587.0,6.0,0.999142,0.0,0.0,3,0,0.448276,2019-07-11


In [350]:
# Merging Training and Test set to get feature 'VOLUME_AVG'

data_test1 = data_train.merge(data_test,how="right",on = ['CAPACITY_BUCKET_ID','CATEGORY_ID','dow'])

# Removing duplicates

data_test1 = data_test1.drop_duplicates(['DATE_TZ_y','CAPACITY_BUCKET_ID','CATEGORY_ID','VOLUME_y','dow','VOLUME_AVG'])

# Dropping unnecessary features 

data_test1.drop(['y_hat_x','VOLUME_x','IsHoliday_x','Is_weekend_x','DATE_TZ_x'],axis = 1,inplace = True)

# Renaming the columns
data_test1.rename(columns = {'DATE_TZ_y':'DATE_TZ', 'VOLUME_y':'VOLUME','y_hat_y':'y_hat','IsHoliday_y':'IsHoliday','Is_weekend_y':'Is_weekend'},inplace = True)

# Sorting the dataframe datewise
data_test1 = data_test1.sort_values('DATE_TZ')

# Setting the index
data_test1.index = data_test1['DATE_TZ']

# Dropping the date field
data_test1.drop('DATE_TZ',axis = 1,inplace = True)

# Creating new feature
data_test1['Predicted'] = 0

print(data_test1.shape)
data_test1.head()

(42594, 9)


,CAPACITY_BUCKET_ID,CATEGORY_ID,dow,VOLUME_AVG,y_hat,VOLUME,IsHoliday,Is_weekend,Predicted
DATE_TZ,,,,,,,,,
2019-12-16,300851.0,5.0,3,17.172414,20.136377,23.0,0.0,0,0
2019-12-16,300853.0,6.0,3,1.310345,1.769150,0.0,0.0,0,0
2019-12-16,300898.0,5.0,3,2.000000,3.547779,1.0,0.0,0,0
2019-12-16,310416.0,6.0,3,1.448276,1.914918,1.0,0.0,0,0
2019-12-16,304197.0,5.0,3,22.034483,33.219769,22.0,0.0,0,0


In [351]:
# Prediction

X_test = data_test1.drop(['VOLUME','Predicted'],axis = 1)
y_test = data_test1['VOLUME']

data_test1['Predicted'] = cat_model.predict(X_test)

In [355]:
# Exporting Results

data_test1.to_csv('SARIMAX_Results8.csv')

In [353]:
# Calculating Metrics

def metrics_calci(source, is_df = False):
   
    # Importing the dataset as is or from a csv file
    if is_df: 
        df = source
    else:
        df = pd.read_csv(source)
        
    # Calculating metrics for each observation    
    df['mape'] = abs(df['VOLUME']+1-df['Predicted'])/(df['VOLUME']+1) * 100 # We are using adjusted MAPE as actual_count can be 0 sometimes
    df['dsigma_num'] = abs(df['VOLUME']-df['Predicted'])
    df['dsigma_den'] = (abs(df['VOLUME'])+abs(df['Predicted']))
    df['dsigma_smape'] = df['dsigma_num'] / df['dsigma_den'] * 100
    df['smape'] = abs(df['VOLUME']-df['Predicted'])/(abs(df['VOLUME'])+abs(df['Predicted'])) * 100
    df['smape2'] = 2*abs(df['VOLUME']-df['Predicted'])/(abs(df['VOLUME'])+abs(df['Predicted'])) * 100
    df['mse'] = (df['VOLUME']-df['Predicted'])**2
    df['rmse'] = abs(df['VOLUME']-df['Predicted'])
    df['mae'] = abs(df['VOLUME']-df['Predicted'])
    
    # Calculating metrics for the whole dataset
    MAPE = df['mape'].mean()
    DSIGMA_SMAPE = df['dsigma_num'].mean() * 100 /df['dsigma_den'].mean()
    SMAPE = df['smape'].mean() 
    SMAPE2 = df['smape2'].mean()
    MSE = df['mse'].mean()
    RMSE = df['mse'].mean() ** 0.5
    MAE = df['mae'].mean()
    
    # Results for the whole dataset
    print('Following are the calculated metrics for the dataset:')
    print('MAPE : %.3f' % MAPE)
    print('RMSE : %.3f' % RMSE)
    print('SMAPE : %.3f' % SMAPE)
    print('MAE : %.3f' % MAE)
    print('MSE : %.3f' % MSE)
    print('DSIGMA_SMAPE : %.3f' % DSIGMA_SMAPE)
    print('SMAPE2 : %.3f' % SMAPE2)

    # Calculating metrics by grouping up geo_id & cat_id  
    df_metrics = pd.DataFrame(df.groupby(['CAPACITY_BUCKET_ID','CATEGORY_ID']).agg({'mape':['mean'],'dsigma_num':['mean'],'dsigma_den':['mean'],'smape':['mean'],'smape2':['mean'],'mae':['mean'],'mse':['mean']})).reset_index()
    df_metrics.columns = ['CAPACITY_BUCKET_ID','CATEGORY_ID','MAPE','DSigma_SMAPE_NUM','DSigma_SMAPE_DEN','SMAPE','SMAPE2','MAE','MSE']
    df_metrics['DSigma_SMAPE'] = df_metrics['DSigma_SMAPE_NUM'] / df_metrics['DSigma_SMAPE_DEN'] *100
    df_metrics['RMSE'] = df_metrics['MSE']**0.5
    
    # Preparing the grouped metrics dataframe 
    df_metrics = df_metrics.drop(['DSigma_SMAPE_NUM', 'DSigma_SMAPE_DEN'], axis=1)
    df_metrics = df_metrics[['CAPACITY_BUCKET_ID','CATEGORY_ID','MAPE','RMSE','SMAPE','MAE','MSE','DSigma_SMAPE','SMAPE2']]
    
    # Preparing the original dataframe
    df = df.drop(['dsigma_num', 'dsigma_den'], axis=1)
    df = df[['DATE_TZ','CAPACITY_BUCKET_ID','CATEGORY_ID','mape','rmse','smape','mae','mse','dsigma_smape','smape2']]

    return df, df_metrics

In [356]:
source = 'SARIMAX_Results8.csv'
raw, grouped = metrics_calci(source, is_df = False)

Following are the calculated metrics for the dataset:
MAPE : 28.223
RMSE : 1.442
SMAPE : 25.093
MAE : 1.091
MSE : 2.078
DSIGMA_SMAPE : 4.625
SMAPE2 : 50.186


In [46]:
data_sarima = pd.read_csv('/jup/AS/Dispatch_Forecasting/SARIMAX_Results2.csv')

In [53]:
data_CDO = pd.read_csv('/jup/AS/Dispatch_Forecasting/CDO_Predicted20191216.csv')

In [47]:
data_sarima.shape

(42594, 9)

In [48]:
data_sarima.head()

,DATE_TZ,DATE_TZ.1,CAPACITY_BUCKET_ID,CATEGORY_ID,VOLUME,Predicted,IsHoliday,dow,Is_weekend
0,2019-12-16,2019-12-16,300851.0,5.0,12.0,20.772971,0.0,0,0
1,2019-12-16,2019-12-16,300851.0,6.0,3.0,1.554038,0.0,0,0
2,2019-12-16,2019-12-16,300851.0,7.0,7.0,13.271161,0.0,0,0
3,2019-12-16,2019-12-16,300852.0,5.0,4.0,4.457111,0.0,0,0
4,2019-12-16,2019-12-16,300852.0,6.0,0.0,0.149019,0.0,0,0


In [54]:
data_CDO.shape

(41850, 5)

In [55]:
data_CDO.head()

,Unnamed: 0,CAPACITY_BUCKET_ID,CATEGORY_ID,prediction,DATE_TZ
0,6833810,300851.0,5.0,18.44,2019-12-16
1,6833693,300851.0,6.0,4.29,2019-12-16
2,6833020,300851.0,7.0,15.53,2019-12-16
3,6402600,300852.0,5.0,5.13,2019-12-16
4,6402483,300852.0,6.0,1.46,2019-12-16


In [56]:
data_CDO.drop('Unnamed: 0',axis = 1,inplace = True)

In [49]:
data_sarima.drop('DATE_TZ.1',axis = 1,inplace = True)

In [50]:
data_sarima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42594 entries, 0 to 42593
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DATE_TZ             42594 non-null  object 
 1   CAPACITY_BUCKET_ID  42594 non-null  float64
 2   CATEGORY_ID         42594 non-null  float64
 3   VOLUME              42594 non-null  float64
 4   Predicted           42594 non-null  float64
 5   IsHoliday           42594 non-null  float64
 6   dow                 42594 non-null  int64  
 7   Is_weekend          42594 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 2.6+ MB


In [51]:
data_sarima['DATE_TZ'] = pd.to_datetime(data_sarima['DATE_TZ'])

In [59]:
merge = data_CDO['CAPACITY_BUCKET_ID'].unique().tolist()
merge

[300851.0,
 300852.0,
 300853.0,
 300854.0,
 300855.0,
 300856.0,
 300857.0,
 300858.0,
 300859.0,
 300860.0,
 300861.0,
 300862.0,
 300863.0,
 300864.0,
 300865.0,
 300866.0,
 300867.0,
 300868.0,
 300869.0,
 300870.0,
 300871.0,
 300872.0,
 300873.0,
 300874.0,
 300875.0,
 300876.0,
 300877.0,
 300878.0,
 300879.0,
 300880.0,
 300881.0,
 300882.0,
 300883.0,
 300884.0,
 300885.0,
 300886.0,
 300887.0,
 300888.0,
 300889.0,
 300890.0,
 300891.0,
 300892.0,
 300893.0,
 300894.0,
 300895.0,
 300896.0,
 300897.0,
 300898.0,
 300899.0,
 300900.0,
 300901.0,
 300902.0,
 300903.0,
 300904.0,
 300905.0,
 300906.0,
 300907.0,
 300908.0,
 300909.0,
 300910.0,
 300911.0,
 300912.0,
 300913.0,
 300914.0,
 300915.0,
 300916.0,
 301575.0,
 301576.0,
 301577.0,
 301579.0,
 301580.0,
 301581.0,
 301582.0,
 301583.0,
 301584.0,
 301585.0,
 301586.0,
 301587.0,
 301588.0,
 301589.0,
 301590.0,
 301591.0,
 301592.0,
 301593.0,
 301594.0,
 301595.0,
 301596.0,
 301597.0,
 301598.0,
 301599.0,
 301600.0,

In [61]:
sarima = len(data_sarima['CAPACITY_BUCKET_ID'].unique().tolist())
sarima

450

In [ ]:
k = [i for i in sarima if i not in merge]
k

In [60]:
data_sarima = data_sarima.loc[(data_sarima['CAPACITY_BUCKET_ID'] != 312861) & (data_sarima['CAPACITY_BUCKET_ID'] != 312867) & (data_sarima['CAPACITY_BUCKET_ID'] != 313680) & (data_sarima['CAPACITY_BUCKET_ID'] != 313720) & (data_sarima['CAPACITY_BUCKET_ID'] != 313762) & (data_sarima['CAPACITY_BUCKET_ID'] != 314201) & (data_sarima['CAPACITY_BUCKET_ID'] != 314300) & (data_sarima['CAPACITY_BUCKET_ID'] != 314362)]

In [62]:
data_sarima.shape

(41850, 8)

In [63]:
data_sarima.to_csv('SARIMAX_Results2.csv')